# Esercitazione SemEval

In [55]:
import hashlib
import numpy as np
import re
import csv
from scipy.stats import pearsonr, spearmanr
from scipy.spatial.distance import cosine
import os.path
import os

### Consegna 1 - Annotazione di coppie di parole 

L'annotazione consiste in un punteggio di semantic similarity di 50 coppie di termini.

Il criterio da utilizzare è presente al link https://tinyurl.com/y6f8h2kd. 

In particolare: 
- 4: **Molto simili** -- Le parole sono sinonimi (e.g., midday-noon). 
- 3: **Simili** -- Le parole condividono gran parte delle idee di significato ma includono dettagli profondamente differenti (e.g., lion-zebra). 
- 2: **Leggermente simili** -- Le parole non hanno significato molto simile ma condividono un argomento/dominio/funzione/idee/concetti che sono correlati (e.g., house-window). 
- 1: **Differenti** -- Le parole descrivono chiaramente concetti differenti, ma condividono qualche piccolo dettaglio come una lontana relazione o un dominio di utilizzo simile in un documento (e.g., software-keyboard).
- 0: **Totalmente differenti e scorrelati** -- Le due parole non significano la stessa cosa e riguardano argomenti differenti (e.g., pencil-frog).

1.1 Estrazione di 50 coppie a partire dal cognome "Scarpinati"

In [30]:
def get_range(surname: str) -> int:
    nof_elements = 500
    base_idx = (abs(int(hashlib.sha512(surname.encode('utf-8')).hexdigest(), 16)) % 10)
    idx_intervallo = base_idx * 50+1
    return idx_intervallo
 

input_name = "Scarpinati"

values = []
sx = get_range(input_name)
values.append(sx)
dx = sx+50-1
intervallo = "" + str(sx) + "-" + str(dx)
print('{:15}:\tcoppie nell\'intervallo {}'.format(input_name, intervallo))


Scarpinati     :	coppie nell'intervallo 251-300


In [31]:
path_corpus = f"utils/it.test.data.txt"

with open(path_corpus, 'r', encoding='utf8') as file:
    reader = file.readlines()[sx:dx+1]
    pairs = [line.strip().split("\t") for line in reader]
    print(pairs)
    print(len(pairs))

[['crittogramma', 'simbolo'], ['campus', 'università'], ['libertà', 'libertà'], ['tifone', 'ciclone'], ['manuale', 'guida turistica'], ['Ebola', 'virus Ebola'], ['principessa', 'biscotto'], ['treno', 'tram'], ['ecosistema', 'economia'], ['regina regnante', 'regina degli scacchi'], ['oro', 'zinco'], ['cursore', 'patatine fritte'], ['Rio delle Amazzoni', 'foresta'], ['tennis', 'statistica'], ['Bolzano', 'teorema'], ['altitudine', 'conversione'], ['barca', 'albero'], ['trono', 'spada'], ['cotone', 'maglione'], ['ciliegia', 'fragola'], ['Islam', 'Corano'], ['Neanderthal', 'sport'], ['nepotismo', 're'], ['personaggio', 'persona'], ['cisterna', 'vagone'], ['Mercurio', 'Giove'], ['bronchite', 'acetaminofene'], ['sangue', 'corpo'], ['islamofobia', 'ISIS'], ['uncinetto', 'uniforme'], ['Hadoop', 'touchscreen'], ['combustibile fossile', 'fossile'], ['mutante', 'sociobiologia'], ['sorella', 'fratello'], ['matita', 'storia'], ['eczema', 'dermatite'], ['imperatore', 'governatore'], ['corona', 'chiav

1.2 Memorizzazione delle coppie estratte

**ATTENZIONE!** NON ESEGUIRE PIU' QUESTA CELLA, IL FILE E' GIA' STATO CREATO

In [32]:
path_extracted = f"resources/it.test.data_extracted.tsv"
if not os.path.exists(path_extracted):
    with open(path_extracted, 'w', newline='', encoding='utf8') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(["Term1", "Term2"])
        tsv_writer.writerows(pairs)

1.3 Estrazione dei valori di similarità inseriti per ogni coppia e calcolo della media

In [34]:
similarity_livio = []
similarity_laura = []
mean_similarity = []

path_annotated = f"resources/it.test.data_annotated.tsv"

with open(path_annotated, 'r', encoding='utf8') as file:
    tsv_file = csv.reader(file, delimiter="\t")
    header = next(tsv_file)
    for line in tsv_file:
        similarity_livio.append(float(line[2]))
        similarity_laura.append(float(line[3]))

mean_similarity = [(similarity_livio[i] + similarity_laura[i])/2 for i in range(50)]

print(similarity_livio)
print(similarity_laura)
print(mean_similarity)

print(len(similarity_laura))
print(len(similarity_livio))
print(len(mean_similarity))

[3.0, 3.0, 4.0, 4.0, 2.0, 4.0, 0.0, 3.0, 2.0, 2.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0, 3.0, 3.0, 3.0, 2.0, 0.0, 2.0, 2.0, 0.0, 3.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 3.0, 0.0, 1.0, 1.0, 0.0, 3.0, 3.0, 0.0, 0.0, 1.0, 0.0, 3.0, 3.0, 2.0]
[2.5, 3.2, 4.0, 3.7, 3.0, 4.0, 0.2, 3.5, 2.3, 2.3, 3.2, 0.0, 2.5, 0.5, 0.0, 0.0, 1.2, 2.5, 3.2, 3.3, 3.0, 0.2, 2.2, 2.6, 1.2, 3.2, 1.5, 3.3, 1.2, 1.3, 0.1, 2.2, 0.1, 3.6, 0.3, 2.7, 3.3, 0.2, 3.0, 3.0, 1.2, 3.7, 3.4, 3.0, 1.3, 1.2, 1.0, 3.2, 3.7, 1.5]
[2.75, 3.1, 4.0, 3.85, 2.5, 4.0, 0.1, 3.25, 2.15, 2.15, 3.1, 0.0, 2.75, 0.25, 0.0, 0.0, 1.1, 2.75, 3.1, 3.15, 2.5, 0.1, 2.1, 2.3, 0.6, 3.1, 1.25, 2.65, 1.1, 1.15, 0.05, 1.1, 0.05, 3.3, 0.15, 2.35, 3.15, 0.1, 2.0, 2.0, 0.6, 3.35, 3.2, 1.5, 0.65, 1.1, 0.5, 3.1, 3.35, 1.75]
50
50
50


1.4 Memorizzazione del documento con il valore medio di valutazione per ogni coppia

In [36]:
os.remove(path_annotated)

with open(path_annotated, 'w', newline='', encoding='utf8') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Term1", "Term2", "Sim_Livio", "Sim_Laura", "Mean"])
    for pair, sim_livio, sim_laura, mean in zip(pairs, similarity_livio, similarity_laura, mean_similarity):
        tsv_writer.writerow([pair[0], pair[1], sim_livio, sim_laura, mean])

1.5 Calcolare agreement fra gli annotatori (inter-rater agreement)

Vengono utilizzati gli indici di correlazione di **Pearson** e **Spearman**

In [43]:
print("##### Inter Agreement with Pearson & Spearman:")
pearson = pearsonr(similarity_livio, similarity_laura)
spearman = spearmanr(similarity_livio, similarity_laura)
print("Pearson ")
print("Coefficiente di correlazione:", pearson[0])
print("p-value: ", pearson[1])
print("\n\nSpearman ")
print("Coefficiente di correlazione:", spearman.correlation)
print("p-value: ", spearman.pvalue)

##### Inter Agreement with Pearson & Spearman:
Pearson 
Coefficiente di correlazione: 0.8608464960885513
p-value:  1.0915876501062841e-15


Spearman 
Coefficiente di correlazione: 0.8717448999587988
p-value:  1.7525719905898133e-16


1.6 Valutazione

La valutazione dei punteggi annotati è in rapporto alla similarità ottenuta utilizzando i vettori NASARI. 

In [52]:
def get_nasari_vectors() -> dict: 
    path_nasari = f"utils/mini_NASARI.tsv"
    nasari_vectors = {}
    with open(path_nasari, 'r', encoding='utf8') as file:
        tsv_file = csv.reader(file, delimiter="\t")
        for line in tsv_file:
            synset_id, synset_name = line[0].split("__")
            vector = [float(value) for value in line[1:]]
            nasari_vectors[synset_id] = vector
    return nasari_vectors

def get_senses2synsets() -> dict: 
    path_senses2synsets = f"utils/SemEval17_IT_senses2synsets.txt"
    senses2synsets = {}
    with open(path_senses2synsets, 'r', encoding='utf8') as file:
        reader = file.readlines()
        i = 0
        while i < len(reader):
            if reader[i].startswith("#"):
                synset_name = reader[i][1:].strip()
                synset_ids = []
                i += 1
                while i < len(reader) and not reader[i].startswith("#"):
                    synset_ids.append(reader[i].strip())
                    i += 1
                senses2synsets[synset_name] = synset_ids
    return senses2synsets

def get_synsets(word: str, nasari_vectors: dict, senses2synsets: dict) -> list:
    synsets = []
    if word in senses2synsets:
        for synset_id in senses2synsets[word]:
            if synset_id in nasari_vectors:
                synsets.append(nasari_vectors[synset_id])
    return synsets


In [56]:
def get_cosine_similarity(v1: list, v2: list) -> float:
    return 1 - cosine(v1, v2)
 

In [63]:
nasari_vectors = get_nasari_vectors()
senses2synsets = get_senses2synsets()

similarities_nasari = []
for word1, word2 in pairs: 
    print(word1, word2)
    synsets1 = get_synsets(word1, nasari_vectors, senses2synsets)
    synsets2 = get_synsets(word2, nasari_vectors, senses2synsets)
    if len(synsets1) == 0 or len(synsets2) == 0:
        print("No synsets found for at least one of the words")
        similarities_nasari.append(0)
        continue
    max_similarity = 0
    for s1 in synsets1:
        for s2 in synsets2:
            similarity = get_cosine_similarity(s1, s2)
            if similarity > max_similarity:
                max_similarity = similarity
    similarities_nasari.append(round(max_similarity, 4))
    print("Similarity:", round(max_similarity, 4))

crittogramma simbolo
Similarity: 0.624
campus università
Similarity: 0.9408
libertà libertà
Similarity: 1
tifone ciclone
Similarity: 1
manuale guida turistica
Similarity: 0.8345
Ebola virus Ebola
Similarity: 0.993
principessa biscotto
Similarity: 0.3956
treno tram
Similarity: 0.8679
ecosistema economia
Similarity: 0.4969
regina regnante regina degli scacchi
No synsets found for at least one of the words
oro zinco
Similarity: 0.8507
cursore patatine fritte
Similarity: 0.3544
Rio delle Amazzoni foresta
Similarity: 0.6174
tennis statistica
Similarity: 0.6604
Bolzano teorema
Similarity: 0.5703
altitudine conversione
Similarity: 0.7004
barca albero
Similarity: 0.9279
trono spada
Similarity: 0.5891
cotone maglione
Similarity: 0.6254
ciliegia fragola
Similarity: 0.9073
Islam Corano
Similarity: 0.8843
Neanderthal sport
Similarity: 0.5034
nepotismo re
Similarity: 0.654
personaggio persona
Similarity: 1
cisterna vagone
Similarity: 0.5377
Mercurio Giove
Similarity: 1
bronchite acetaminofene
Simil

1.7 Coefficienti di correlazione tra la media degli annotatori e i risultati di NASARI

In [64]:
def normalize_range(values: list, min: int, max: int) -> list: 
    values_norm = [min + (max - min) * val for val in values]
    return values_norm

mean_norm = normalize_range(mean_similarity, 0, 1)

print("##### Intra Agreement with Pearson & Spearman:")

pearson = pearsonr(mean_norm, similarities_nasari)
spearman = spearmanr(mean_norm, similarities_nasari)

print("Pearson ")
print("Coefficiente di correlazione:", pearson[0])
print("p-value: ", pearson[1])
print("\n\nSpearman ")
print("Coefficiente di correlazione:", spearman.correlation)
print("p-value: ", spearman.pvalue)

##### Intra Agreement with Pearson & Spearman:
Pearson 
Coefficiente di correlazione: 0.4934492329834736
p-value:  0.0002713485798726492


Spearman 
Coefficiente di correlazione: 0.5742331662269193
p-value:  1.2988340617971147e-05
